<a href="https://colab.research.google.com/github/marthaakinlolu/Atari_Games/blob/master/Pacman.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gym
!pip install gym[atari]
!pip install opencv-python
!pip install gymnasium
!pip install tensorflow
!pip install atari-py
!pip install stable-baselines3
!pip install stable-baselines3[extra]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.8/953.8 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.6/540.6 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for atari-py: filename=atari_py-0.2.9-cp310-cp310-linux_x86_64.whl size=2870285 sha256=c0a5e3307a8cb2806de655cf19f93c99d6bebd3c808721b73f30ca960be8c325
  Stored in directory: /root/.cache/pip/wheels/75/6f/04/1f3bf5255580101e16ff487564354dddcdd23ec3b43b775b7a
Successfully built atari-py
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.4/178.4 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 925.5/925.5 kB 10.0 MB/s eta 0:00:00
  Attempting uninstall: gymnasium
    Found existing installation: gymnasium 0.29.0
    Uninstalling gymnasium-0.29.0:
      Successfully uninstalled gymnasium-0.29.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 9.4 MB/s eta 0:00:00
  Inst

In [ ]:
import os
import gymnasium as gym
import numpy as np
from PIL import Image
from stable_baselines3 import DQN
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
from stable_baselines3.dqn.policies import CnnPolicy
from gymnasium.utils.save_video import save_video
from gymnasium.wrappers import FrameStack,  ResizeObservation

/usr/local/lib/python3.10/dist-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(


In [ ]:
env = gym.make("MsPacman-v4")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Get the observation space
observation_space = env.observation_space

# Check the shape of the observation space
print("Observation space shape:", observation_space.shape)
print("Observation space data type:", observation_space.dtype)

Observation space shape: (210, 160, 3)
Observation space data type: uint8


In [ ]:
# Get the action space
action_space = env.action_space

# Check the number of possible actions
num_actions = action_space.n
print("Number of possible actions:", num_actions)

Number of possible actions: 9


In [ ]:
class DQNAgent:
    def __init__(self, name=None, env_name=None, eval_freq=20000, buffer_size=1000):
        """
        Initialize the DQNAgent.

        Parameters:
            name (str): Name of the game.
            env_name (str): Environment name.
            eval_freq (int): Evaluation frequency.
            buffer_size (int): Buffer size for the replay buffer.
        """
        self.name = name
        self.env_name = env_name
        self.eval_freq = eval_freq
        self.buffer_size = buffer_size
        self.log_path = os.path.join('Training/DQN_' + self.name + '_Log')
        self.save_path = os.path.join('Saved_Models/DQN_' + self.name + '_Model')
        self.env = self.make_environment()
        self.model = self._build_dqn()

    def make_environment(self):
        """
        Create the environment.

        Returns:
            gym.Env: The created environment.
        """
        return gym.make(self.env_name, render_mode='rgb_array')

    def _build_dqn(self):
        """
        Build the DQN model.

        Returns:
            stable_baselines3.DQN: The DQN model.
        """
        model = DQN(CnnPolicy, self.env, verbose=0, tensorboard_log=self.log_path, buffer_size=self.buffer_size)
        return model

    def _play_one_episode(self):
        """
        Play one episode of the game.

        Returns:
            float: The total score accumulated during the episode.
        """
        obs, _ = self.env.reset()
        done = False
        score = 0

        while not done:
            action = self.env.action_space.sample()
            obs, reward, done, *info = self.env.step(action)
            score += reward

        return score

    def play_episodes(self, num_episodes=10, play_type="random"):
        """
        Play episodes of the game.

        Parameters:
            num_episodes (int): Number of episodes to play.
            play_type (str): The type of play - "random" or "predict".
        """
        if play_type == "random":
            print(f"Playing the {self.name} game randomly for {num_episodes} episodes")
            scores = [self._play_one_episode() for _ in range(num_episodes)]
            for episode, score in enumerate(scores, 1):
                print(f"Episode {episode}: {score}")

        if play_type == "predict":
            episode_rewards = []
            frames = []

            for episode in range(num_episodes):
                obs, _ = self.env.reset()
                done = False
                score = 0

                while not done:
                    action, _ = self.model.predict(obs)
                    obs, reward, done, *info= self.env.step(action)
                    score += reward
                    frame = Image.fromarray(self.env.render())
                    frame = np.array(frame)
                    frames.append(frame)

                episode_rewards.append(score)
                print(f"Episode {episode+1}: {score}")

            video_path = os.path.join(self.save_path, self.name + "_Agent_play")
            save_video(frames, video_path, fps=30, name_prefix=f"{self.name}-agent-play")

    def train(self, time_steps=None, stop_value=None):
        """
        Train the agent.

        Parameters:
            time_steps (int): Total number of training steps.
            stop_value (float): Reward threshold to stop training early.
        """
        stop_callback = StopTrainingOnRewardThreshold(reward_threshold=stop_value, verbose=0)
        eval_callback = EvalCallback(self.env, callback_on_new_best=stop_callback,
                                     eval_freq=self.eval_freq, best_model_save_path=self.save_path)
        self.model.learn(total_timesteps=time_steps, callback=eval_callback)

    def evaluate_policy(self, episodes=None):
        """
        Evaluate the policy learned by the agent.

        Parameters:
            episodes (int): Number of episodes to evaluate.
        """
        mean_reward, reward_std = evaluate_policy(self.model, self.env, n_eval_episodes=episodes)
        print(f"Mean reward over {episodes} episodes is {mean_reward} with a standard deviation of {reward_std}")

    def load_best_model(self):
        """
        Load the best model saved during training.

        Returns:
            stable_baselines3.DQN: The best model.
        """
        return DQN.load(self.save_path + "/best_model")

    def save_model(self):
        """
        Save the trained DQN model.
        """
        return self.model.save(self.save_path)

    def close_env(self):
        """
        Close the environment.
        """
        self.env.close()


In [ ]:
# Create a DQNAgent for the MsPacman environment
MsPacman_agent = DQNAgent(name="MsPacman", env_name="MsPacman-v4")

In [ ]:
# Play 5 episodes randomly
MsPacman_agent.play_episodes(num_episodes=20)

Playing the MsPacman game randomly for 20 episodes
Episode 1: 210.0
Episode 2: 230.0
Episode 3: 150.0
Episode 4: 190.0
Episode 5: 190.0
Episode 6: 250.0
Episode 7: 300.0
Episode 8: 250.0
Episode 9: 200.0
Episode 10: 160.0
Episode 11: 120.0
Episode 12: 220.0
Episode 13: 260.0
Episode 14: 210.0
Episode 15: 270.0
Episode 16: 180.0
Episode 17: 350.0
Episode 18: 170.0
Episode 19: 220.0
Episode 20: 280.0


In [9]:
# Train the agent for 10000 time steps
MsPacman_agent.train(time_steps=100000, stop_value=1000)

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/callbacks.py:403: UserWarning: Training and eval env are not of the same type<stable_baselines3.common.vec_env.vec_transpose.VecTransposeImage object at 0x79690be753c0> != <stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv object at 0x79690be75f30>
  warnings.warn("Training and eval env are not of the same type" f"{self.training_env} != {self.eval_env}")
/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=20000, episode_reward=60.00 +/- 0.00
Episode length: 618.40 +/- 5.20
New best mean reward!
Eval num_timesteps=40000, episode_reward=60.00 +/- 0.00
Episode length: 625.80 +/- 10.24
Eval num_timesteps=60000, episode_reward=802.00 +/- 372.10
Episode length: 840.60 +/- 143.05
New best mean reward!
Eval num_timesteps=80000, episode_reward=374.00 +/- 130.32
Episode length: 709.20 +/- 78.85
Eval num_timesteps=100000, episode_reward=692.00 +/- 477.05
Episode length: 803.40 +/- 212.78


In [10]:
# Evaluate the policy after training
MsPacman_agent.evaluate_policy(episodes=10)

Mean reward over 10 episodes is 419.0 with a standard deviation of 219.88406035909014


In [ ]:
# Save the trained DQN model
MsPacman_agent.save_model()

In [12]:
# Play 10 episodes using the trained policy
MsPacman_agent.play_episodes(num_episodes=10, play_type="predict")

/usr/local/lib/python3.10/dist-packages/gymnasium/utils/passive_env_checker.py:364: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(


Episode 1: 480.0
Episode 2: 580.0
Episode 3: 210.0
Episode 4: 270.0
Episode 5: 430.0
Episode 6: 720.0
Episode 7: 200.0
Episode 8: 470.0
Episode 9: 1310.0
Episode 10: 490.0
Moviepy - Building video /content/Saved_Models/DQN_MsPacman_Model/MsPacman_Agent_play/MsPacman-agent-play-episode-0.mp4.
Moviepy - Writing video /content/Saved_Models/DQN_MsPacman_Model/MsPacman_Agent_play/MsPacman-agent-play-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /content/Saved_Models/DQN_MsPacman_Model/MsPacman_Agent_play/MsPacman-agent-play-episode-0.mp4


In [ ]:
# Load the best model (if training with evaluation callback)
best_model = MsPacman_agent.load_best_model()

In [ ]:
# Close the environment
MsPacman_agent.close_env()